<a href="https://colab.research.google.com/github/Yong-Zhuang/Deep-Learning-Analysis-of-Altered-Sensorimotor-Representations-of-the-Body-in-Spinal-Cord-Injury/blob/main/sensorimotor_representations_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/python

from math import sqrt
import sys;
import csv;
import numpy as np;
from math import *;
from sklearn.linear_model import LinearRegression

dataPath = sys.argv
print(dataPath[1])

data = open(str(dataPath[1]), 'r')
file = csv.DictReader(data)

model = LinearRegression()


def getRodVector(rodVals, linmodel):
    #print(rodVals)
    vals = [float(val) for val in rodVals]
    # print(vals)
    x = [val for idx, val in enumerate(vals) if idx % 2 == 0]
    y = [val for idx, val in enumerate(vals) if idx % 2 == 1]
    npx = np.asarray(x).reshape((-1, 1))
    npy = np.asarray(y)
    linmodel.fit(npx,npy)
    rxo = x[0]
    ryo = rxo * linmodel.coef_[0] + linmodel.intercept_
    rxt = x[-1]
    ryt = rxt * linmodel.coef_[0] + linmodel.intercept_
    return [linmodel.coef_[0], linmodel.intercept_]

def checkPawMiss(rodVals, frontPVals):
    x = [float(val) for idx, val in enumerate(frontPVals) if idx % 2 == 0]
    y = [float(val) for idx, val in enumerate(frontPVals) if idx % 2 == 1]
    pawmiss = []
    for pawIdx in range(0, len(x)):
        # print(type(rodVals[0]))
        # print(rodVals[1])
        # print(type(x[0]))
        # print(y[0])
        if rodVals[0]*x[pawIdx] + rodVals[1] < y[pawIdx] : #assuming 0, 0 is topleft
            pawmiss.append(True)
        else:
            pawmiss.append(False)
    for missed in pawmiss:
        if missed:
            return True
    return False


def checkPawOnRung(rodVals, pawVals):
    px = [float(val) for idx, val in enumerate(pawVals) if idx % 2 == 0]
    py = [float(val) for idx, val in enumerate(pawVals) if idx % 2 == 1]
    rx = [float(val) for idx, val in enumerate(rodVals) if idx % 2 == 0]
    ry = [float(val) for idx, val in enumerate(rodVals) if idx % 2 == 1]


    pcx = 0.0
    pcy = 0.0
    for pawIdx in range(0, len(px)):
        pcx += px[pawIdx]
        pcy += py[pawIdx]

    pcx /= len(px)
    pcy /= len(py)

    isOnOrCloseToRung = False
    #print("Checking dists")
    whichRod = -1
    for rodIdx in range(0, len(rx)):
        dist = sqrt(pow(pcx-rx[rodIdx], 2) + pow(pcy-ry[rodIdx], 2))
        #print(dist)
        if (dist <= 15):
            isOnOrCloseToRung = True
            whichRod = rodIdx
    return (isOnOrCloseToRung, whichRod)

jointData = {}
linenum = 0
for row in file:
    if linenum == 0:
        for header in row:
            jointData[header] = []
        print(jointData)
        linenum += 1
    
    for header in row:
        jointData[header].append(row[header])
    linenum += 1

vals = jointData.values()
dataLen = len(list(vals)[0])
score = 0


touchedRungPrevFrame = False
touchedRungPrev = -1
curTouchedRung = -1
numSlipped = 0
numShift = 0
numFull = 0 
numPartialBackPaw = 0
numMiss = 0
for t in range(0, dataLen):
    #print(t)
    rodVals = []
    frontPawVals = []
    backPawVals = []
    for valIdx, rowVal in enumerate(vals):
        if valIdx >= 12 and valIdx <= 17:
            frontPawVals.append(rowVal[t])
        if valIdx >= 18 and valIdx <= 23:
            backPawVals.append(rowVal[t])
        if valIdx >= 24:
            rodVals.append(rowVal[t])
    
    rodLine = getRodVector(rodVals, model)

    # check if paw miss
    if not checkPawMiss(rodLine, frontPawVals):
        checkRes = checkPawOnRung(rodVals, frontPawVals)
        if checkRes[0]:
            touchedRungPrevFrame = True
            curTouchedRung = checkRes[1]
            
           # print("front on rung")
            if checkPawOnRung(rodVals, backPawVals):
                #print("back also on rung")
                #print("perfect frame")
                if touchedRungPrev != -1:
                    if curTouchedRung != touchedRungPrev:
                        score += 2
                        numShift += 1
                    else:
                        score += 4
                    touchedRungPrev = curTouchedRung
                else:
                    score += 4
                    touchedRungPrev = curTouchedRung
                numFull += 1
        #else:
            #print("front in air")

    else:
        backOnRung = checkPawOnRung(rodVals, backPawVals)
        #print("front missed")
        if touchedRungPrevFrame:
            score += 1
            numSlipped += 1
            #print("slipped")
        if backOnRung:
            score += 3
            numPartialBackPaw += 1
            #print("back on rung")
        numMiss += 1
        touchedRungPrevFrame = False
    # print(rodLine)

print("Score: " + str(score))
print("Missed: " + str(numMiss))
print("Slipped: " + str(numSlipped))
print("Front Shift: " + str(numShift))
print("PartialBack: " + str(numPartialBackPaw))
print("Perfect: " + str(numFull))
